In [1]:
import os
import z_function_list
from functools import reduce

In [2]:
base_path,data,outputs = z_function_list.folder_setup()

In [3]:
os.chdir(data)

In [4]:
#pull in excel file
cleanstat = pd.ExcelFile('clean_comparison.xlsx')
#create dictionary of frames for each sheet
d = {n:cleanstat.parse(n) for n in cleanstat.sheet_names}
#merge into single DF

for item in list(d.keys()):
    cols = list(d[item].columns)
    cols = [x+"_"+item if x!='SegmentID' else 'SegmentID' for x in cols]
    d[item].columns = cols
    
#combine into single df
combined_quarters = reduce(lambda x,y: pd.merge(x,y,on='SegmentID'),[d[sheet] for sheet in cleanstat.sheet_names])

#don't think I actually need this anymore
#missing_cd_map = {k:v for k,v in pd.read_csv('clean_stat_missing_CD.csv')[['SegmentID','District']].values}

#create new universal columns
combined_quarters['Miles']=combined_quarters['Miles_Q1_2016']
combined_quarters['CD']=combined_quarters['CD_Q1_2018']
combined_quarters['FullName']=combined_quarters['FullName_Q4_2017']

#ID fields that contain score
scores = list(filter(lambda x: 'CS_RoundSc' in x,list(combined_quarters.columns)))

combined_quarters_simple_columns = combined_quarters[['CD','Miles','SegmentID']+scores].copy()



In [8]:
combined_quarters[['SegmentID','Miles']].set_index(['SegmentID'])['Miles']*5280

SegmentID
1           8.112364
2         115.233494
3           6.258701
5         230.512549
7          52.428997
9         101.811435
10       1215.217149
13        233.231925
15         81.716110
16         88.803782
19        996.097232
20        665.996236
31         24.984783
32         52.407657
33         20.018032
42        787.413472
43        782.526869
44        901.106756
45        361.096190
46        570.053244
47        569.793768
49         47.058978
51         69.511135
52         31.167781
53        532.067626
54        667.070192
55         56.046764
56         44.119083
57         30.902645
59        612.901875
62         49.797176
68         17.489111
69         26.075445
70         23.752751
72         68.339118
73         38.674944
77        731.124896
78        156.404652
79        739.291110
80        273.390240
81        389.598145
82        490.659230
86        791.625081
87         25.701932
90         36.660836
91         70.601400
100       212.484281
101

In [244]:
duplicate_segment_ID = list(combined_quarters['SegmentID'].value_counts()[combined_quarters['SegmentID'].value_counts()>1].index)

In [222]:
os.chdir(data)
latest_quarter = 'Q1_2018'
combined_quarters_simple_columns.to_csv('Streets_Index'+'_'+'Q1_2016_to'+latest_quarter)

In [223]:
def pivot_maker(score_field):
    pivot = combined_quarters_simple_columns.pivot_table('Miles',index=score_field,
                                                        columns='CD',aggfunc='sum')
    pivot['All'] = pivot.sum(axis=1)
    pivot['South LA'] = pivot[[8,9,10]].sum(axis=1)
    pivot['East LA'] = pivot[[1,14]].sum(axis=1)
    pivot['Southeast Valley']=pivot[2]
    pivot['Northeast Valley'] = pivot[[6,7]].sum(axis=1)
    pivot['West Valley'] = pivot[[3,12]].sum(axis=1)
    pivot['Hollywood/South Valley']= pivot[4]
    pivot['Westside']= pivot[[5,11]].sum(axis=1)
    pivot['Harbor'] = pivot[15]
    pivot['Silver Lake/Echo Park/Hollywood'] = pivot[13]
    pivot = pivot[['All','South LA','East LA','Southeast Valley','West Valley','Northeast Valley',
                 'Hollywood/South Valley','Westside','Harbor','Silver Lake/Echo Park/Hollywood']].T
    return pivot.rename(columns={0:'Not Evalauted',1:'Clean',2:'Somewhat Clean',3:'Not Clean'})

In [224]:
summary_dict = {x:pivot_maker(x) for x in scores}

In [255]:
np.round(summary_dict['CS_RoundSc_Q1_2018'].loc['Northeast Valley'],8)

CS_RoundSc_Q1_2018
Clean             1007.937634
Somewhat Clean      26.597664
Not Clean            6.850514
Name: Northeast Valley, dtype: float64

In [213]:
d["Q1_2016"]['SegmentID'].value_counts()

2047     1
3467     1
7561     1
28039    1
25990    1
32133    1
30084    1
19843    1
17794    1
23937    1
21888    1
42366    1
46460    1
36219    1
34170    1
40313    1
38264    1
11631    1
9582     1
1418     1
13708    1
7593     1
15757    1
28071    1
26022    1
32165    1
30116    1
19875    1
17826    1
23969    1
21920    1
44447    1
42398    1
46492    1
36251    1
34202    1
40345    1
38296    1
11663    1
9614     1
15725    1
13676    1
3435     1
7529     1
1354     1
7497     1
5448     1
27975    1
25926    1
32069    1
        ..
41697    1
8945     1
10992    1
37607    1
39654    1
33509    1
35556    1
45795    1
17085    1
29371    1
43648    1
31418    1
29339    1
31386    1
25241    1
27288    1
4759     1
6806     1
2708     1
12947    1
14994    1
8849     1
10896    1
37511    1
39558    1
33413    1
35460    1
45699    1
41601    1
19100    1
17053    1
23198    1
8881     1
25273    1
27320    1
4791     1
6838     1
2740     1
12979    1
15026    1

In [ ]:
#### q1_2016_ = combined_quarters.pivot_table('Miles',columns='Cat_Q1_2016',index='CD',aggfunc='sum')[['Clean', 'Somewhat Clean', 'Not Clean', 'Not Evaluated']]
q4_2016_ = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2016',index='CD',aggfunc='sum')[['Clean','Somewhat Clean','Not Clean']]
q4_2017_ = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2017',index='CD',aggfunc='sum')[['Clean','Somewhat Clean','Not Clean']]

q1_2016 = combined_quarters.pivot_table('Miles',columns='Cat_Q1_2016',index='Updated CD',aggfunc='sum')[['Clean', 'Somewhat Clean', 'Not Clean', 'Not Evaluated']]
q4_2016 = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2016',index='Updated CD',aggfunc='sum')[['Clean','Somewhat Clean','Not Clean']]
q4_2017 = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2017',index='Updated CD',aggfunc='sum')[['Clean','Somewhat Clean','Not Clean']]

In [366]:
q1_2016_SouthLA = q1_2016.loc[[8,9,10]]
q1_2016_SouthLA_ = q1_2016_.loc[[8,9,10]]

q4_2016_SouthLA = q4_2016.loc[[8,9,10]]
q4_2016_SouthLA_ = q4_2016_.loc[[8,9,10]]

q4_2017_SouthLA = q4_2017.loc[[8,9,10]]
q4_2017_SouthLA_ = q4_2017_.loc[[8,9,10]]

In [387]:
## ALL LA

all_recoded = reduce(lambda x,y: pd.merge(x,y,left_index=True,right_index=True),
                            [q1_2016,q4_2016,q4_2017]).T

all_no_0 = reduce(lambda x,y: pd.merge(x,y,left_index=True,right_index=True),
                            [q1_2016_,q4_2016_,q4_2017_]).T

#need to remove the 0 districts!
all_no_0 = all_no_0.drop(0,axis=1)

#create new index

updated_index = [x.replace('_y','_Q4_2016') for x in [x.replace('_x','_Q1_2016') for x in all_recoded.index]]
updated_index[3] = 'Not_Evaluated_Q1_2016'
updated_index[-3:]=[x+'_Q4_2017' for x in updated_index[-3:]]


## SOUTH LA

#recoded districts
south_LA_recoded = reduce(lambda x,y: pd.merge(x,y,left_index=True,right_index=True),
                            [q1_2016_SouthLA,q4_2016_SouthLA,q4_2017_SouthLA]).T

#original districts, subtracting "0" districts
south_LA_no_0 = reduce(lambda x,y: pd.merge(x,y,left_index=True,right_index=True),
                            [q1_2016_SouthLA_,q4_2016_SouthLA_,q4_2017_SouthLA_]).T


all_recoded.index,all_no_0.index, south_LA_no_0.index, south_LA_recoded.index = [updated_index]*4



In [302]:
def percent_changes(df,starting,ending):
    raw = (df.sum(axis=1).loc[ending]-df.sum(axis=1).loc[starting])/(df.sum(axis=1).loc[starting])
    return(format(raw*100,'.1f')+'% change')

In [378]:
percent_changes(south_LA_no_0,ending='Clean_Q4_2017',starting='Clean_Q1_2016')

'110.7% change'

In [379]:
percent_changes(south_LA_recoded,ending='Clean_Q4_2017',starting='Clean_Q1_2016')

'119.9% change'

In [388]:
percent_changes(all_no_0,ending='Clean_Q4_2017',starting='Clean_Q1_2016')

'54.4% change'

In [389]:
percent_changes(all_recoded,ending='Clean_Q4_2017',starting='Clean_Q1_2016')

'69.6% change'

In [216]:
os.chdir(outputs)
combined_quarters.to_csv('cleanstat_Q1Q42016_Q42017.csv')